### Baseline
rating = α ∗ μi + β ∗ ηj

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# import the data covert that into a convenient data
PATH= '/home/ac-optimus/data_science/recommendation_sys/ratings_small.csv'
df = pd.read_csv(PATH, sep= ",")
# df = pd.read_csv(PATH, sep= ",")
df.shape

(100004, 4)

In [5]:
df.drop('timestamp', axis=1, inplace=True)
df.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [6]:
# this gave me a good plot
from sklearn.utils import shuffle
df = shuffle(df)

In [7]:
df.head()

,userId,movieId,rating
44975,315,1278,5.0
37532,270,5952,5.0
64158,461,349,2.0
47090,345,8784,4.0
98750,664,111,4.0


In [8]:
num_rows = df.shape[0]
train, test = df.copy(deep = True), df[int(num_rows*0.8):].copy(deep = True)
train[int(num_rows*0.8):]['rating']  = train[int(num_rows*0.8):]['rating']*0
train = shuffle(train)

<ipython-input-8-75ff325d5135>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[int(num_rows*0.8):]['rating']  = train[int(num_rows*0.8):]['rating']*0


In [9]:
user_movie_rating =train.groupby(['userId','movieId'])['rating'].max().unstack().fillna(0)
user_movie_rating.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def error(y_hat, y):
    # MSE
    y_hat = np.array(y_hat)
    y = np.array(y)
    MSE_ = (y_hat- y)**2
    MSE = MSE_.sum()/ len(y_hat)
    return MSE

In [12]:
# avg_user_rating(μi) - the average rating of user i
# avg_movie_rating(ηj) - that of movie j over all ratings  
# α and β are fitted using training data. I am using linear regression for this.

avg_user_rating = user_movie_rating.mean(axis=1).tolist()
avg_movie_rating = user_movie_rating.mean(axis=0).tolist()
len(avg_movie_rating), len(avg_user_rating)

(9066, 671)

In [13]:
# mapping movieid, userid and corresponding average ratings.
columns = user_movie_rating.columns.tolist()
indexes = user_movie_rating.index.tolist()

new_df =  df.copy(deep=True)
new_df['movie_avg_rating'] = new_df['movieId'].apply(lambda x:avg_movie_rating[columns.index(x)])
new_df['user_movie_rating'] = new_df['userId'].apply(lambda x:avg_user_rating[indexes.index(x)])
new_df.head()

,userId,movieId,rating,movie_avg_rating,user_movie_rating
44975,315,1278,5.0,0.374814,0.013126
37532,270,5952,5.0,0.865872,0.073186
64158,461,349,2.0,0.538748,0.171741
47090,345,8784,4.0,0.260805,0.069821
98750,664,111,4.0,0.633383,0.175491


In [14]:
# baesline test and train datasets. It is different from the above test train test split. 
# Follows the usual train-test split
baseline_train = new_df[:int(num_rows*0.8)].drop(['userId', 'movieId'], axis=1)
baseline_test =  new_df[int(num_rows*0.8):].drop(['userId', 'movieId'], axis=1)
baseline_test.shape, baseline_train.shape

((20001, 3), (80003, 3))

In [25]:
%%time
# looking for optimal k
y= test['rating']
errors = {}
y_hat1 = []
misses = 0
for i in test.index:
    user_i, movie_j = map(int,[test.loc[i]['userId'], 
                               test.loc[i]['movieId']])
    if movie_j in baseline_train['movie_avg_rating']:
        predicted_rating = baseline_train['movie_avg_rating'][movie_j]
    else:
        misses+=1
        predicted_rating = 0
    y_hat1.append(predicted_rating)
error_i = error(np.nan_to_num(np.array(y_hat1)), y)        


CPU times: user 6.77 s, sys: 4.72 ms, total: 6.78 s
Wall time: 6.81 s


### MSE

In [20]:
error_i = error(np.nan_to_num(np.array(y_hat1)), y)        
error_i

12.17350406117035

#### Number of test pairs for which there is not prediction

In [21]:
misses

4209